In [1]:
import pandas as pd
import numpy as np
import cvxportfolio as cvx
from datetime import datetime, timedelta

In [2]:
companies = ['GAZP', 'SBER', 'LKOH', 'GMKN', 'MGNT',  'TATN', 'NVTK', 'SNGS', 'PLZL', 'SNGSP',
         'PIKK', 'ROSN', 'SBERP', 'CHMF', 'NLMK', 'IRAO', 'YNDX', 'ALRS', 'RUAL', 'MTSS','MAGN', 
         'PHOR', 'RTKM', 'VTBR', 'TATNP', 'TCSG', 'AGRO', 'OZON', 'AFLT', 'AFKS','FEES', 'TRNFP',
         'CBOM', 'VKCO', 'ENPG', 'SGZH', 'FIVE',  'MOEX', 'GLTR', 'HYDR', 'FIXP']

In [3]:
def expected_returns(data, day, hour, companies):
    '''
    Принимает на вход запрос по дате времени и активу
    Возвращает предстказанную доходность
    '''
    output = np.zeros(len(companies))
    preds = data.loc[day, hour]
    availible_companies = preds.Asset.values
    
    for i, company in enumerate(companies):
        if company in availible_companies:
            output[i] = preds[preds.Asset == company].pct_pred[0]
        else:
            output[i] = 0
    return output

def real_returns(data, day, hour, companies):
    '''
    Принимает на вход запрос по дате времени и активу
    Возвращает реальную доходность
    '''
    output = np.zeros(len(companies))
    preds = data.loc[day, hour]
    availible_companies = preds.Asset.values
    
    for i, company in enumerate(companies):
        if company in availible_companies:
            output[i] = preds[preds.Asset == company].pct_fact[0]
        else:
            output[i] = 0
    return output 

def hour2time(hour):
    if hour == 1:
        res = '11:00:00'
    elif hour == 2:
        res = '12:00:00'
    elif hour == 3:
        res = '13:00:00'
    elif hour == 4:
        res = '14:00:00'
    elif hour == 5:
        res = '15:00:00'
    elif hour == 6:
        res = '16:00:00'
    elif hour == 7:
        res = '17:00:00'
    elif hour == 8:
        res = '18:00:00'
    elif hour == 9:
        res = '18:45:00'
    return res

In [67]:
returns = pd.read_csv('cvxportfolio_pred_returns', index_col='Date')
returns.index = pd.DatetimeIndex(returns.index).tz_localize('Europe/Moscow')

closes = pd.read_csv('cvxportfolio_fact_closes', index_col='Date')
closes.index = pd.DatetimeIndex(closes.index).tz_localize('Europe/Moscow')

volumes = pd.read_csv('volumes', index_col='Date')
volumes.index = closes.index 

In [71]:
objective = cvx.ReturnsForecast(r_hat=returns) - cvx.StocksTransactionCost(a=0.005)

In [72]:
constraints = [cvx.LongOnly(applies_to_cash=True)]

In [73]:
policy = cvx.SinglePeriodOptimization(objective, constraints)

In [58]:
# data = cvx.UserProvidedMarketData(returns=my_forecast, prices=None)

In [76]:
simulator = cvx.MarketSimulator(returns=returns,  min_history=pd.Timedelta('100 days 05:45:36'))

In [77]:
results = simulator.backtest(policy, start_time='2023-02-03 11:00:00', end_time='2023-10-24 18:45:00')

DataError: HistoricalMeanVolume can only be used if MarketData provides market volumes.